# Classification Metrics with Seattle Rain

Predicting whether or not it will rain in Seattle tomorrow, based on whether or not it rained in Seattle yesterday, is a classic classification problem (moreover, a classic [Markov chain](https://en.wikipedia.org/wiki/Markov_chain) simulation problem, but I digress...).

To measure how well we do on classification tasks like this one we take advantage of a well-developed variety of *classification metrics*. These compliment the regression metrics I covered in [a previous notebook](https://www.kaggle.com/residentmario/model-fit-metrics/).

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("../input/seattleWeather_1948-2017.csv")
df.head()

In [ ]:
df = df.dropna()
X = df.loc[:, ['PRCP', 'TMAX', 'TMIN']].shift(-1).iloc[:-1].values
y = df.iloc[:-1, -1:].values.astype('int')

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X, y)
y_hat = clf.predict(X)

## Accuracy score

The accuracy of a binary classifier is nothing less and nothing more than the proportion of predictions which are made correctly. In this case, our model was accurate ~70% of the time.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_hat)

Accuracy is usually the end result, and what the model end user will see and care about. However, it's not very useful for diagnosing a classifier because it doesn't tell us *where* the model is making errors. Answering this "where" question is an essential part of model-building. Hence accuracy is a start, but not an end.

## Confusion matrix

The confusion matrix steps beyond the accuracy score by providing us with information about where we are committing errors, in the form of a conditional distribution table.

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y, y_hat)

It's usually convenient to plot the confusion matrix in a `seaborn` `heatmap`:

In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix(y, y_hat) / len(y), cmap='Blues', annot=True)

The 0,0 and 1,1 cells are correct classifications, and their sum is the classifier accuracy. These are also known as the *true positive* and *true negative* rates. The 1,0 cell is *false positive rate*: times when the classifier generates a false positive (in our case, predicting that it will rain when it doesn't). This occurs in 13% of all cases. Meanwhile, 17% of all cases is *false negative rate* (predicting it will not rain when it does) in the 0,1 cell.

In a multiclass setting, this plot will expand out with more entries, but the story will remain the same: results on the diagonal are good, results off the diagonal are bad, and finding where those off-diagonal results tend to occur will help us understand where our errors are and how we can improve on them.

In this case we see that our false negative rate is significantly higher (relative to our true negative rate) than our false positive rate: 0.17 is a bigger with respect to 0.26 than 0.13 is to 0.44. Perhaps (at least with more complicated data!) we should investigate what errors we are making in those misclassifications.

## Hamming loss

Hamming loss or [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) is a pretty cool concept taken from information theory. In information theory, the Hamming distance between two pieces of information is the number of bits of the recieved signal that we would need to change in order to match the signal sent. Another thing to think about it is as the distance between a word and a misspelling of that word, e.g. "animus" and "animos" would have a Hamming loss of 1, since we have to make one correction to match the two words.

In classification tasks, however, Hamming loss simplifies down to $1 - \text{Accuracy}$, as the following demonstrates (recall that our prediction accuracy was 70%).

In [ ]:
from sklearn.metrics import hamming_loss

hamming_loss(y, y_hat)

## Precision and recall; F-score

**Precision** and **recall** are two metrics which are often tied together by a computed fit statistic called the **F-score**.

Precision and recall originate from the field of information retrieval. In information retrieval, we are tasked with providing users with records which are relevant to tehir search query, and not records which are irrelevant. Suppose, for example, that we are running a search engine. Our search engine returns 30 pages, of which 20 are relevant and 10 are irrelevant. Our search engine also fails to return 40 other relevant pages.

In this toy example, precision is the percentage of results we've returned which are relevant: 20/30, or 2/3. Recall, meanwhile, the "completeness" of the information we've returned with respect to all possible relevant results: 20/60, or 1/3.

In math:

$$\text{Precision} = \frac{\text{True Positives}}{\text{True Positives + False Positives}}$$
$$\text{Recall} = \frac{\text{True Positives}}{\text{True Positives + False Negatives}}$$

Here's a picture that Wikipedia provides for the concepts:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/440px-Precisionrecall.svg.png)

If you read any literature on search, you will learn that finding the right balance between precision and recall is an important part of search. Lightweight users care less about recall and more about precision, because they are mainly interested in finding a few hits that satisfy their lightweight needs. An example of a search fitting this profile might be "Can dogs eat chocolate?". Power users meanwhile prioritize recall; they can tolerate a few false positives, so long as they are able to thoroughly research "General Zhukov's role in the Stalingrad counterattack".

At the same time, precision and recall are fundamentally a trade-off. Returning *all* of the relevant results naturally requires returning at least *some* edge cases that turn out to be useless, and returning *only* relevant results naturally requires excluding *some* edge cases that turn out to be useful.

In [ ]:
from sklearn.metrics import precision_score, recall_score, precision_recall_curve

print(precision_score(y, y_hat))
print(recall_score(y, y_hat))

`sklearn` includes a utility for visualizing the trade-off between these two values, the `precision_recall_curve`.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

precision, recall, _ = precision_recall_curve(y, y_hat)

fig, ax = plt.subplots(1, figsize=(12, 6))
ax.step(recall, precision, color='steelblue',
         where='post')
ax.fill_between(recall, precision, step='post', color='lightgray')
plt.suptitle('Precision-Recall Tradeoff for Seattle Rain Prediction')
plt.xlabel('Recall')
plt.ylabel('Precision')

In the classification context, this trade-off between precision and recall is often encoded using the **F-score**. An F-score is a fit statistic, scored between 0 and 1, which measures model fitness as a weighted [harmonic mean](https://en.wikipedia.org/wiki/Harmonic_mean) of precision and recall. Which of these two measures matters more is controlled by the $\beta$ hyperparameter. A $\beta$ of 0 considers only precision (and so is equivalent to the `precision_score`, a $\beta$ of 1 is known as an **F1 score** and considers an even balance of the two, while F1 scores greater than 1 (to infinity) are weighted more heavily towards recall.

What should your choice of $\beta$ be? The F1 score is worth pointing out (and even has its own `sklearn` implementation, `f1_score`) because it is a reasonable default. Choosing a specific ratio of precision to recall is usually a domain decision. We should favor a higher $\beta$ if recall is more important for our model end users, and a lower $\beta$ if precision is more important to our model end users.

Keep in mind that it just so happens that F1, precision, and recall are often very targeted at the model end user (e.g. they correspond well with the classification model *as a product*). If these measures are not meaningful to us or our product, then we shouldn't be using them at all.

In [ ]:
from sklearn.metrics import fbeta_score

fbeta_score(y, y_hat, beta=1)

## ROC curve

The **ROC curve** is the most common graphic way of examining the performance of a model. It stands for "Reciever Operator Characteristic" (dunno why), and it plots the performance of the model on two measures: the *true positive rate* and the *false positive rate*. Here is what a ROC curve looks like:

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y, y_hat)

fig, ax = plt.subplots(1, figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', label='Model Performace')
plt.plot([0, 1], [0, 1], color='gray', label='Random Performace')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Seattle Rain ROC Curve')
plt.legend(loc="lower right")

The ROC curve is a nice and heavily utilized classification metric because it does a good job measuring how much harder getting incrementally more correct responses is for the model.

To get the first few true positives, the model does not have to work very hard, because we can pick up records which we have a very high confidence in. The model's confidence in each of the results that we collect afterwards drops steadily; picking up more true positives also involves making some errors, and hence, picking up more false positives. The last few results will naturally be the hardest to squeeze out; they will involve making the largest number of errors (picking up the most false positives).

A model which picks up only true positives reliably will have a very high slope on as much of the line as possible, probably only leveling out on the last few records. Hence, when it comes to the ROC curve, the best model is one which sticks as close to the top of the chart as possible.

By looking a different points on the ROC curve, we can diagnose how much confidence the model has in different parts of the data. For example, if the ROC curve for the first quartile is extremely steep, then we see that the model "clears" the first 25% of true positives very easily. Seeing where the model begins to "fall off" is often very informative, because it's those records that we can often make most progress accuracy on (and hardly ever more marginal records further down the curve!).

## AUC

The ROC curve can be summarized using a single metric: the "area under the curve", or AUC. AUC is an oft-used metric of choice for classification model parameter tuning. It is no more and no less than the area under the curve above (an integral, or perhaps a trapazoidal approximation thereof).

A higher AUC is better: more area under the curve, and hence, less input to the false positive rate. This metric is implemented in `sklearn` thusly:

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y, y_hat)

# Conclusion

These metrics are the basics of measuring classification performace. There are other more advanced measurements, like the Gini coefficient and log-loss, which I will cover later in other notebooks.

Hopefully you found this material informative!